In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [7]:
#Problem 1
#logistic regression

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  l2_beta = tf.constant(beta)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + l2_beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 46.654579
Training accuracy: 11.3%
Validation accuracy: 12.8%
Loss at step 100: 11.705710
Training accuracy: 73.4%
Validation accuracy: 71.9%
Loss at step 200: 4.424366
Training accuracy: 78.8%
Validation accuracy: 76.6%
Loss at step 300: 1.955882
Training accuracy: 82.4%
Validation accuracy: 79.9%
Loss at step 400: 1.119116
Training accuracy: 84.0%
Validation accuracy: 81.1%
Loss at step 500: 0.832251
Training accuracy: 84.6%
Validation accuracy: 81.8%
Loss at step 600: 0.732680
Training accuracy: 84.8%
Validation accuracy: 82.1%
Loss at step 700: 0.697784
Training accuracy: 84.9%
Validation accuracy: 82.1%
Loss at step 800: 0.685462
Training accuracy: 84.8%
Validation accuracy: 82.2%
Test accuracy: 88.8%


In [13]:
#now add regularization to SGD 1 layer neural network

num_nodes = 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([num_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(relu, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2)) + beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  valid_logit1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  valid_relu = tf.nn.relu(valid_logit1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_relu, weights_2) + biases_2)
  test_logit1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  test_relu = tf.nn.relu(test_logit1)
  test_prediction = tf.nn.softmax(
    tf.matmul(test_relu, weights_2) + biases_2)

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3468.859375
Minibatch accuracy: 12.5%
Validation accuracy: 26.6%
Minibatch loss at step 500: 21.131901
Minibatch accuracy: 87.5%
Validation accuracy: 84.1%
Minibatch loss at step 1000: 0.950222
Minibatch accuracy: 82.0%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 0.577893
Minibatch accuracy: 88.3%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.605092
Minibatch accuracy: 90.6%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 0.714722
Minibatch accuracy: 83.6%
Validation accuracy: 83.4%
Minibatch loss at step 3000: 0.767140
Minibatch accuracy: 82.8%
Validation accuracy: 83.3%
Test accuracy: 90.3%


In [12]:
#Problem 2
#Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?
num_steps = 3001
small_train_dataset = train_dataset[0:384,] #batch size is 128, this should restrict to ~3 batches
small_train_labels = train_labels[0:384,]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

#high minibatch accuracy reflects overfitting

Initialized
Minibatch loss at step 0: 3515.564941
Minibatch accuracy: 9.4%
Validation accuracy: 33.1%
Minibatch loss at step 500: 21.047314
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%
Minibatch loss at step 1000: 0.443913
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 1500: 0.279635
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 2000: 0.267471
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 2500: 0.261262
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 3000: 0.257481
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Test accuracy: 84.2%


In [14]:
#Problem 3
#Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.

num_nodes = 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([num_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu_output = tf.nn.relu(logits_1)
  dropout_relu = tf.nn.dropout(relu_output, keep_prob = 0.5)
  logits_2 = tf.matmul(dropout_relu, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2)) + beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  valid_logit1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  valid_relu = tf.nn.relu(valid_logit1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_relu, weights_2) + biases_2)
  test_logit1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  test_relu = tf.nn.relu(test_logit1)
  test_prediction = tf.nn.softmax(
    tf.matmul(test_relu, weights_2) + biases_2)

#call the neural network with dropout
num_steps = 3001
small_train_dataset = train_dataset[0:384,] #batch size is 128, this should restrict to ~3 batches
small_train_labels = train_labels[0:384,]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_dataset[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

#high minibatch accuracy reflects overfitting

Initialized
Minibatch loss at step 0: 3626.825684
Minibatch accuracy: 6.2%
Validation accuracy: 30.2%
Minibatch loss at step 500: 21.028229
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 0.467604
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 0.297530
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 0.286141
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 2500: 0.284269
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 3000: 0.278098
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Test accuracy: 84.3%
